<a href="https://colab.research.google.com/github/goranagojic/paralelno-racunarstvo/blob/main/PR_termin2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Paralelno računarstvo - Vežba 2

Vežba obuhvata zadatke:
*    Zadatak 1 - Sekvencijalno množenje matrice i vektora
*    Zadatak 2 - Paralelizacija množenja matrice i vektora - blokovsko raspoređivanje
*    Zadatak 3 - Paralelizacija množenja matrice i vektora - ciklično raspoređivanje
*    Zadatak 4 - Paralelizacija množenja matrice i vektora - blok-ciklični raspored

## Preduslovi

Pre nego započnete implementaciju zadatka, **obavezno** izvršite kod iz ćelije ispod. Kod će:

*   napraviti direktorijum u koji će se čuvati sve izvrne i izvršne datoteke
*   preuzeti datoteku `hpc_helpers.hpp` i sačuvati je na putanju /home/naprednicpp/

In [ ]:
%%bash

# napravi radni direktorijum
mkdir -p /home/naprednicpp

# preuzmi hpc_helper zaglavlje
[[ -f hpc_helpers.zip ]] || wget -O hpc_helpers.zip "https://drive.google.com/u/0/uc?id=1PeEfh8h5SjEcQ5154Z6TpHIlJogQODwM&export=download"
unzip -oqqd /home/naprednicpp/ hpc_helpers.zip

## Zadatak 1

Neka je $A$ matrica dimenzija $m \times n$, a $x$ vektor dužine $n$. Potrebno je izračunati vektor $b$ koji je proizvod matrice $A$ i vektora $x$ tj. $b = Ax$.
$$
b_{i} = \sum_{j=0}^{n-1}A_{ij}\cdot x_{j}, \forall i \in \{ 0, \dots, m-1 \}
$$
Sekvencijalno rešenje implementirati u telu funkcije `sequential_mult` u postavci zadatka (videti ispod).

## Zadatak 2

Paralelizovati rešenje za množenje matrice i vektora iz prethodnog zadatka koristeći primitive klase `thread`. Pri implementaciji koristiti blokovski raspored niti i lambda funkcije.

## Zadatak 3

Paralelizovati rešenje za množenje matrice i vektora koristeći primitive klase `thread`. Implementirati ciklično raspoređivanje niti uz korišćenje lambda funkcija.

## Zadatak 4

Paralelizovati rešenje za množenje matrice i vektora koristeći primitive klase `thread`. Implementirati blok-ciklični raspored niti i lambda funkcije.

In [ ]:
%%writefile /home/naprednicpp/mnozenjematricavektor.cpp

#include "hpc_helpers.hpp"

#include <iostream>
#include <cstdint>
#include <vector>
#include <thread>

template <
typename value_t,
typename index_t>
void init(

    std::vector<value_t>& A,
    std::vector<value_t>& x,
    index_t m,
    index_t n) {
    for (index_t row = 0; row < m; row++)
        for (index_t col = 0; col < n; col++)
            A[row*n+col] = row >= col ? 1 : 0;
    for (index_t col = 0; col < n; col++)
        x[col] = col;

}


template <
    typename value_t,
    typename index_t>
void sequential_mult(
    std::vector<value_t>& A,
    std::vector<value_t>& x,
    std::vector<value_t>& b,
    index_t m,
    index_t n) {
        
    // TODO ovde implementirati resenje zadatka 1

}


template <
    typename value_t,
    typename index_t>
void cyclic_parallel_mult(
    std::vector<value_t>& A, // linear memory for A
    std::vector<value_t>& x, // to be mapped vector
    std::vector<value_t>& b, // result vector
    index_t m,               // number of rows
    index_t n,               // number of cols
    index_t num_threads=8) { // number of threads p
    
    // TODO ovde implementirati resenje zadatka 2

}


template <
    typename value_t,
    typename index_t>
void block_parallel_mult(
    std::vector<value_t>& A,
    std::vector<value_t>& x,
    std::vector<value_t>& b,
    index_t m,
    index_t n,
    index_t num_threads=32) {

    // TODO ovde implementirati resenje zadatka 3

}


template <
    typename value_t,
    typename index_t>
void block_cyclic_parallel_mult(
    std::vector<value_t>& A,
    std::vector<value_t>& x,
    std::vector<value_t>& b,
    index_t m,
    index_t n,
    index_t num_threads=8,
    index_t chunk_size=64/sizeof(value_t)) {

        // TODO ovde implementirati resenje zadatka 4
}


int main(int argc, char* argv[]) {

    const uint64_t n = 1UL << 2;
    const uint64_t m = 1UL << 2;

    TIMERSTART(overall)
    TIMERSTART(alloc)
    std::vector<  uint64_t> A(m*n);
    std::vector<  uint64_t> x(n);
    std::vector<  uint64_t> b(m);
    TIMERSTOP(alloc)

    TIMERSTART(init)
    init(A, x, m, n);
    TIMERSTOP(init)

    TIMERSTART(mult_seq)
    sequential_mult(A, x, b, m, n);
    TIMERSTOP(mult_seq)

    // TIMERSTART(mult_block)
    // block_parallel_mult(A, x, b, m, n);
    // TIMERSTOP(mult_block)

    // TIMERSTART(mult_cyclic)
    // cyclic_parallel_mult(A, x, b, m, n);
    // TIMERSTOP(mult_cyclic)

    // TIMERSTART(mult_block_cyclic)
    // block_cyclic_parallel_mult(A, x, b, m, n);
    // TIMERSTOP(mult_block_cyclic)

    TIMERSTOP(overall)

    //for (const auto& entry: b)
    //    std::cout << entry << std::endl;

    for (uint64_t index = 0; index < m; index++)
        if (b[index] != index*(index+1)/2)
            std::cout << "error at position " << index << " "
                      << b[index] << std::endl;

}


Overwriting /home/naprednicpp/mnozenjematricavektor.cpp


### Kompajliranje izvornog koda

Pokrenite komandu u ćeliji ispod kako biste kompajlirali rešenje.

In [ ]:
%%bash

cd /home/naprednicpp
g++ hpc_helpers.hpp mnozenjematricavektor.cpp -o mnozenjematricavektor -pthread

### [Opciono] Provera da li postoji izvršna datoteka

In [ ]:
![[ -f /home/naprednicpp/sumavektora ]] && echo "Postoji." || echo "Ne postoji."

Ne postoji.


### Pokretanje rešenja

Pokretanje nemodifikovane postavke rešenja rezultuje ispisima:
*    vremena izvršavanja za različite delove programa - sve funkcije za množenje matrice i vektora bi trebalo da budu izuzetno male vrednosti, jer funkcije nisu implementirane
*    o netačnom rezultatu (*greška na poziciji ...*). Ovi ispisi će nestati nakon što ispravno implementirate funkcije za množenje matrice i vektora. Nakon uspešne implementacije povećajte vrednosti promenljivih $m$ i $n$ (npr. šiftovanjem za 14 pozicija) kako biste videli uticaj raspoređivanja na performanse izvršavanja rešenja.

In [ ]:
%%bash

cd /home/naprednicpp
./mnozenjematricavektor